In [2]:
import pandas as pd
from pathlib import Path

# Set relative path 
BASE_DIR = Path.cwd()
output_runs_path = BASE_DIR / "data" / "runs_matrix.csv"
output_outs_path = BASE_DIR / "data" / "outs_matrix.csv"

# Define states and events
states = [f"{outs}{first}{second}{third}"
          for outs in range(3)
          for first in range(2)
          for second in range(2)
          for third in range(2)]

events = [
    'DoublePlay', 'Error', 'GroundOut', 'HBP/CatInt', 'HR', 'K',
    'LineOut/InfFly', 'Triple', 'Walk', 'LongSingle',
    'MediumSingle', 'ShortSingle', 'ShortDouble', 'LongDouble', 'LongFly',
    'MediumFly', 'ShortFly'
]

# Create "event_matrix" function
def event_matrix(current_state, event):
    outs = int(current_state[0])
    on_1b, on_2b, on_3b = map(int, current_state[1:])
    runs = 0
    new_outs = outs

    if event == 'HR':
        runs += on_1b + on_2b + on_3b + 1
        on_1b, on_2b, on_3b = 0, 0, 0

    elif event == 'Walk':
        if on_1b == 0:
            on_1b = 1
        elif on_2b == 0:
            on_2b = 1
        elif on_3b == 0:
            on_3b = 1
        else:
            runs += 1

    elif event == 'HBP/CatInt':
        if on_1b == 0:
            on_1b = 1
        elif on_2b == 0:
            on_2b = 1
        elif on_3b == 0:
            on_3b = 1
        else:
            runs += 1

    elif event == 'DoublePlay':
        if current_state in ['0000', '0001', '0010', '0011', '1000', '1001', '1010', '1011', '2000', '2001', '2010', '2011']:
            new_outs = outs + 1 
        
        else:
            new_outs += 2  
            if outs == 0 and on_1b == 1 and on_2b == 0 and on_3b == 1:  
                runs += 1 
            on_1b, on_2b, on_3b = 0, 0, 0 


    elif event == 'Triple':
        runs += on_1b + on_2b + on_3b
        on_1b, on_2b, on_3b = 0, 0, 1

    elif event == 'LongSingle':
        runs += on_3b + on_2b
        on_3b, on_2b, on_1b = on_1b, 0, 1

    elif event == 'MediumSingle':
        runs += on_3b + on_2b
        on_3b, on_2b, on_1b = 0, on_1b, 1

    elif event == 'ShortSingle':
        runs += on_3b
        on_3b, on_2b, on_1b = on_2b, on_1b, 1

    elif event == 'ShortDouble':
        runs += on_3b + on_2b
        on_3b, on_2b, on_1b = on_1b, 1, 0

    elif event == 'LongDouble':
        runs += on_3b + on_2b + on_1b
        on_3b, on_2b, on_1b = 0, 1, 0

    elif event == 'GroundOut':
        new_outs += 1
        if current_state not in ['0011', '0111', '1011']:
            if (outs == 0 and on_3b == 1) or (outs == 1 and on_3b == 1):
                runs += 1
                on_3b = 0
                
    elif event == 'Error':
        runs += on_3b
        on_3b, on_2b, on_1b = on_2b, on_1b, 1

    elif event == 'LongFly':
        if outs < 2:
            runs += on_3b
            on_3b = 0
        new_outs += 1

    elif event == 'MediumFly':
        if outs < 2:
            runs += on_3b
            on_3b = 0
        new_outs += 1

    elif event == 'ShortFly' or event == 'LineOut/InfFly':
        new_outs += 1

    elif event == 'K':
        new_outs += 1

    return runs, min(new_outs - outs, 3)

# Generate Runs Matrix and Outs Matrix
runs_data = []
outs_data = []

for state in states:
    runs_row = []
    outs_row = []
    for event in events:
        runs, outs = event_matrix(state, event)
        runs_row.append(runs)
        outs_row.append(outs)
    runs_data.append(runs_row)
    outs_data.append(outs_row)

runs_matrix = pd.DataFrame(runs_data, index=states, columns=events)
outs_matrix = pd.DataFrame(outs_data, index=states, columns=events)

# Export CSV files
runs_matrix.index = runs_matrix.index.astype(str)
runs_matrix.to_csv(output_runs_path, index_label='current_state')

outs_matrix.index = outs_matrix.index.astype(str)
outs_matrix.to_csv(output_outs_path, index_label='current_state')